In [ ]:
#| default_exp test_end_to_end

In [ ]:
#| hide

from tempfile import TemporaryDirectory
from pathlib import Path
import shutil
from tqdm.auto import tqdm
from PIL import Image
from loguru import logger
import sys

from bedmap.prepare_images import dl_hf_images

from bedmap.config import Cfg


{'image_dir': None, 'image_glob': None, 'thumbnail_size': 128, 'model_name': 'timm/vit_small_patch14_reg4_dinov2.lvd142m', 'umap_config': {'umap_n_components': 10, 'umap_n_neighbors': 15, 'umap_min_dist': 0.1}}


In [ ]:
#| hide

NUM_TEST_IMAGES = 256
TEST_DATASET = "kvriza8/microscopy_images"

# easy timestamps
logger.remove()
logger.add(sys.stdout, level="INFO")

In [ ]:
#| hide

def images_from_paths(pathlist):
    return (Image.open(p.as_posix()).convert("RGB").copy() for p in pathlist)

In [ ]:
#| hide

with TemporaryDirectory() as tmp:
    logger.info("Downloading test images.")
    dl_hf_images(dir=Path(tmp), max_images=NUM_TEST_IMAGES)
    imagepaths = list(Path(tmp).glob("*.png"))
    gen = images_from_paths(imagepaths)
    for i in range(1):
        im = next(gen)
        im.show()
    

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()